# 组合模式

对象能包含体系中的其他对象——组合体，不能包含——非组合体。
组合体和非组合体都具有一套核心方法，但是非组合体还包含增加、移除、遍历的方法。

In [1]:
# 常规的实现方式，组合体和非组合体具有相同的基类、
import abc
import sys


def main():
    pencil = SimpleItem("Pencil", 0.40)
    ruler = SimpleItem("Ruler", 1.60)
    eraser = SimpleItem("Eraser", 0.20)
    pencilSet = CompositeItem("Pencil Set", pencil, ruler, eraser)
    box = SimpleItem("Box", 1.00)
    boxedPencilSet = CompositeItem("Boxed Pencil Set", box, pencilSet)
    boxedPencilSet.add(pencil)
    for item in (pencil, ruler, eraser, pencilSet, boxedPencilSet):
        item.print()


class AbstractItem(metaclass=abc.ABCMeta):

    @abc.abstractproperty
    def composite(self):
        pass


    def __iter__(self):
        return iter([])


class SimpleItem(AbstractItem):

    def __init__(self, name, price=0.00):
        self.name = name
        self.price = price


    @property
    def composite(self):
        return False


    def print(self, indent="", file=sys.stdout):
        print("{}${:.2f} {}".format(indent, self.price, self.name),
                file=file)


class AbstractCompositeItem(AbstractItem):

    def __init__(self, *items):
        self.children = []
        if items:
            self.add(*items)


    def add(self, first, *items):
        self.children.append(first)
        if items:
            self.children.extend(items)


    def remove(self, item):
        self.children.remove(item)


    def __iter__(self):
        return iter(self.children)


class CompositeItem(AbstractCompositeItem):

    def __init__(self, name, *items):
        super().__init__(*items)
        self.name = name


    @property
    def composite(self):
        return True


    @property
    def price(self):
        return sum(item.price for item in self)


    def print(self, indent="", file=sys.stdout):
        print("{}${:.2f} {}".format(indent, self.price, self.name),
                file=file)
        for child in self:
            child.print(indent + "      ")


if __name__ == "__main__":
    main()


$0.40 Pencil
$1.60 Ruler
$0.20 Eraser
$2.20 Pencil Set
      $0.40 Pencil
      $1.60 Ruler
      $0.20 Eraser
$3.60 Boxed Pencil Set
      $1.00 Box
      $2.20 Pencil Set
            $0.40 Pencil
            $1.60 Ruler
            $0.20 Eraser
      $0.40 Pencil


In [2]:
import itertools
import sys


def main():
    pencil = Item.create("Pencil", 0.40)
    ruler = Item.create("Ruler", 1.60)
    eraser = make_item("Eraser", 0.20)
    pencilSet = Item.compose("Pencil Set", pencil, ruler, eraser)
    box = Item.create("Box", 1.00)
    boxedPencilSet = make_composite("Boxed Pencil Set", box, pencilSet)
    boxedPencilSet.add(pencil)
    for item in (pencil, ruler, eraser, pencilSet, boxedPencilSet):
        item.print()
    assert not pencil.composite
    pencil.add(eraser, box)
    assert pencil.composite
    pencil.print()
    pencil.remove(eraser)
    assert pencil.composite
    pencil.remove(box)
    assert not pencil.composite
    pencil.print()


class Item:

    def __init__(self, name, *items, price=0.00):
        self.name = name
        self.price = price
        self.children = []
        if items:
            self.add(*items)


    @classmethod
    def create(Class, name, price):
        return Class(name, price=price)


    @classmethod
    def compose(Class, name, *items):
        return Class(name, *items)

    
    @property
    def composite(self):
        return bool(self.children)


    def add(self, first, *items):
        self.children.extend(itertools.chain((first,), items))


    def remove(self, item):
        self.children.remove(item)


    def __iter__(self):
        return iter(self.children)


    @property
    def price(self):
        return (sum(item.price for item in self) if self.children else
                self.__price)


    @price.setter
    def price(self, price):
        self.__price = price


    def print(self, indent="", file=sys.stdout):
        print("{}${:.2f} {}".format(indent, self.price, self.name),
                file=file)
        for child in self:
            child.print(indent + "      ")


def make_item(name, price):
    return Item(name, price=price)


def make_composite(name, *items):
    return Item(name, *items)


if __name__ == "__main__":
    main()


$0.40 Pencil
$1.60 Ruler
$0.20 Eraser
$2.20 Pencil Set
      $0.40 Pencil
      $1.60 Ruler
      $0.20 Eraser
$3.60 Boxed Pencil Set
      $1.00 Box
      $2.20 Pencil Set
            $0.40 Pencil
            $1.60 Ruler
            $0.20 Eraser
      $0.40 Pencil
$1.20 Pencil
      $0.20 Eraser
      $1.00 Box
$0.40 Pencil
